# QP6

 # Desconsiderando os registros que possuam grande concordância ou pouquíssima concordância

# Tamanho das janelas modificado pela porcentagem

### Modificada a ordenação de dup e ndup a fim de fazer com que as janelas deslizem a partir do meio do conjunto todo e assim pegar pares da região de incerteza

### Orçamento = 200

### Coeficiente de variação <= 5

### Lançando mão da entropia

### Buscando manter o conjunto de treinamento balanceado

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
import os, errno
import re
import numpy as np
import math
import pandas as pd
import sys
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# nao_passaram = 0
# iteracoes = 0

############################################################################################################################
#Definição de funções
############################################################################################################################

#Função para geração do conjunto de treinamento
def geraTrainSet(ct, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
        
    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#Função para geração do conjunto de teste
def geraTestSet(ct, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)            
    

#Função para verificar se o vetor já existe no conjunto de treinamento
def comparaVetor(vetor):
    achou = False

    for index, row in conj_treino.iterrows():

        if vetor.equals(row):
            achou = True
            break

#     print('achou')
#     print(achou)
    
    return achou

#Função para verificar se uma dada parte do vetor (calculada por uma porcentagem)
#já existe no conjunto de treinamento
def comparaVetorPorc(vetor):
    
#     print('****************************')
    
#     print('vetor recebido:')
#     print(vetor)
    
    achou = False

    for index, row in conj_treino.iterrows():
        
        rowX = row[-1*(row.size-1) :] #Desconsiderando aqui a primeira coluna que contém o status do vetor
        vetorX = vetor[-1*(vetor.size-1) :] #Desconsiderando aqui a primeira coluna que contém o status do vetor
        
        try:
            
            vetor_bool = rowX == vetorX #Vetor booleano resultante da comparação do vetor de entrada
                                  #e cada linha do conjunto de treinamento
        
        except:
            
            print("except do comparaVetorPorc")
            
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(sys.exc_info())
        
        cols_iguais = [i for i in vetor_bool if i == True]
        cols_iguais = cols_iguais.count(True) #Quantidade de colunas iguais

        #Se a quantidade de colunas com valores iguais for maior ou igual à dada porcentagem,
        #considera-se que o vetor passado é semelhante a algum já existente no conjunto treino
        if (cols_iguais >= int(0.75*vetor_bool.size)):
            achou = True
            break
        
#     print('achou')
#     print(achou)
    
#     if(not achou):
        
#         print('Vetor distinto dos demais:')
#         print(vetorX)
#         print('qtd de colunas iguais: {0} - 75% seriam: {1}'.format(cols_iguais), int(0.75*vetor_bool.size))
    
    return achou

#Função para verificar a semelhança entre dois vetores dada a distância entre seus valores
def comparaVetorDif(vetor):
    
#     print('****************************')
    
#     print('vetor recebido:')
#     print(vetor)
    
    achou = False

    for index, row in conj_treino.iterrows():
        
        rowX = row[-1*(row.size-1) :] #Desconsiderando aqui a primeira coluna que contém o status do vetor
        vetorX = vetor[-1*(vetor.size-1) :] #Desconsiderando aqui a primeira coluna que contém o status do vetor
        
        try:
            
            vetor_dif = abs(rowX - vetorX) #Vetor com a diferença entre os dois vetores comparados
                                           # (o vetor de entrada e cada linha do conjunto de treinamento)
        except:
            
            print("except do comparaVetorDif")
            
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            print(sys.exc_info())
        
#         print('$$$$$$$$$$$$$$$$$$$$$')
#         print("vetor_dif")
#         print(vetor_dif)
#         print('$$$$$$$$$$$$$$$$$$$$$')
        
        cols = [i for i in vetor_dif if i <= 0.1] #Se a diferença for menor ou igual a 0.1,
                                                        #nesse exemplo, os vetores são considerados iguais
        qtd_cols = len(cols) 

        #Se a quantidade de colunas com valores iguais for maior ou igual à dada porcentagem,
        #considera-se que o vetor passado é semelhante a algum já existente no conjunto treino
        if (qtd_cols >= int(0.90*vetor_dif.size)): #Nesse exemplo, apenas é considerado igual se for maior ou igual a 90%
                                                    #de algum vetor já existente no conjunto de treinamento
            achou = True
            break
#         else:
#             
#             print('$$$$$$$$$$$$$$$$$$$$$')
#             print("vetor_dif")
#             print(vetor_dif)
#             print('$$$$$$$$$$$$$$$$$$$$$')
            
#             print('cols')
#             print(cols)
            
#             print('qtd_cols')
#             print(qtd_cols)
        
#     print('achou')
#     print(achou)
    
#     if(not achou):
        
#         print('Vetor distinto dos demais:')
#         print(vetorX)
#         print('qtd de colunas iguais: {0} - 75% seriam: {1}'.format(cols_iguais), int(0.75*vetor_bool.size))
    
    return achou

#Função para...

#grupo - conjunto de vetores de similaridade de tamanho k
#col_comp - coluna do conjunto "grupo" a ser utilizada para comparação dos vetores deste
#tipo_conj - define se o tratamento do vetor a ser selecionado para rotulação é para o conjunto de possíveis duplicatas ou não-duplicatas

def sel_vet_rot(grupo, col_comp, tipo_conj, qm_chamou):
    warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
    
#     if(qm_chamou == 'rec'):
#         print('A recursividade chamou')
    
#     print(qm_chamou)
#     print("Entrando em sel_vet_rot")
    
    global conj_treino, orcamento, reexecuta, deslz_dup, jan_inic_dup, jan_fin_dup, deslz_ndup, jan_inic_ndup, jan_fin_ndup, k_dup, k_ndup
    
#     print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))
    
#     print("Comprimento do grupo: {0}".format(len(grupo)))
    
    
    if len(grupo) > 0:

        try:
            
            id_row_sel = grupo[col_comp].idxmax() #Retorna o índice da linha com maior valor da coluna ('entropia') especificada

            #SÓ PASSA PRA CÁ SE NÃO LANÇAR EXCEPT

            #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final dacomposição do conjunto de treinamento)
            vetor_sel = pc_vetores.loc[id_row_sel]
#             print('vetor_sel')
#             print(vetor_sel)

            #Se o vetor já existir no conjunto de treino
            if comparaVetorDif(vetor_sel):
#             if comparaVetor(vetor_sel):
#             if comparaVetorPorc(vetor_sel):
                
                grupo.drop([id_row_sel], axis = 'rows', inplace = True)#Retira do grupo o vetor anteriormente selecionado
#                 print("grupo depois de dropado")
#                 print(grupo)
                
                sel_vet_rot(grupo, col_comp, tipo_conj, 'rec')
           
            #Senão, adiciona o vetor selecionado ao conjunto treino	
            else:
                
                conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                orcamento -= 1
                reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                return reexecuta
#                 print("conj_treino")
#                 print(conj_treino)

        except ValueError:

            print("Atualizando janelas no except")
#             exc_type, exc_obj, exc_tb = sys.exc_info()
#             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#             print(exc_type, fname, exc_tb.tb_lineno)
#             print(sys.exc_info())
            
            if tipo_conj == "dup":
                
#                 print("ValueError - tipo_conj == dup")
                
                jan_inic_dup = jan_inic_dup + k_dup
                jan_fin_dup = jan_fin_dup + k_dup

                deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar
            
            elif tipo_conj == "ndup":
                
#                 print("ValueError - tipo_conj == ndup")
                
                jan_inic_ndup = jan_inic_ndup + k_ndup
                jan_fin_ndup = jan_fin_ndup + k_ndup

                deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            reexecuta = True
            return reexecuta
            

        except:

            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
#             print("Eita!")
#             print(sys.exc_info())
# #             print(grupo)
            reexecuta = True
            return reexecuta
#             sys.exit(1)

    #Caso em que todos os vetores já existiam no conj_treino
    #Reexecuta (Desliza a janela e chama sel_vet_rot(----):)
    else:

        print("Atualizando janelas depois de dropar todos os elementos do grupo")
#         print("Quem chamou: {0}".format(qm_chamou))
        
        if tipo_conj == "dup":
                
            jan_inic_dup = jan_inic_dup + k_dup
            jan_fin_dup = jan_fin_dup + k_dup

            deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar
            
        elif tipo_conj == "ndup":
                    
            jan_inic_ndup = jan_inic_ndup + k_ndup
            jan_fin_ndup = jan_fin_ndup + k_ndup

            deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))
        reexecuta = True #Se não selecionou um registro no comando anterior, precisa reexecutar o while
        return reexecuta
        
#     print("reexecuta")
#     print(reexecuta)
    return reexecuta

#Função para geração do F1 médio
def geraF1(toClass, classificador, onde_foi_chamado): #toClass é o conjunto de treinamento
    
#     print("F1 chamado {0}".format(onde_foi_chamado))
    
    f1 = 0
    std = 0
    coef_var = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    
    kfold = KFold(n_splits=2, random_state=seed)
    kfoldUtilizado = "kf-2"
    
    try:
        
#         kfold = StratifiedKFold(n_splits=10, random_state=seed) #Mudar para n-fold #Adicionado depois!
#         kfoldUtilizado = "skf-10"
        
#     except ValueError:
                
        kfold = StratifiedKFold(n_splits=5, random_state=seed)
        kfoldUtilizado = "skf-5"
            
#     except ValueError:
                
#         kfold = KFold(n_splits=10, random_state=seed)
            
    except ValueError:
                
        kfold = KFold(n_splits=5, random_state=seed)
        kfoldUtilizado = "kf-5"
                
    except ValueError:
                
        kfold = KFold(random_state=seed) #Default n_splits = 3
        kfoldUtilizado = "kf-3"
            
    except ValueError:
                
        kfold = KFold(n_splits=2, random_state=seed)
        kfoldUtilizado = "kf-2"
    
    except:
        print('Erro com a geração dos folds!')
        print("kfold utilizado: {0}".format(kfoldUtilizado))
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        print(sys.exc_info())
    
    
    try:
        cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
        
        f1 = cv_results.mean()
        std = cv_results.std()
        coef_var = std/f1
    except:
        print("ERRO NA VALIDAÇÃO CRUZADA!")
        print("kfold utilizado: {0}".format(kfoldUtilizado))
#         print("Conjunto de treinamento:")
#         print(toClass)
        print("Tamanho de toClass: {0}".format(len(toClass)))
        print("F1 foi chamado: {0}".format(onde_foi_chamado))
    
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        print(sys.exc_info())
        
    
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

############################################################################################################################
#Repetições para os experimentos começam aqui
############################################################################################################################

#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'med' #Estatística para ordenamento
#qtd_alg = 23 #Quantidade total de algoritmos (Modificado para ser atualizado em cada iteração de acordo com a quantidade máxima de algoritmos em cada experimento)
qtd_alg_nd = 2 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
# k = 3 #Tamanho da janela
# janelas = [2,3,4,5]
    # janelas = [3] 
janelas = [1, 0.01, 0.03, 0.05] # K = {1, 1%, 3%, 5%}
orcamento = 200 #Ou uma porcentagem da base de dados
orcamento_orig = orcamento
tam_min_ct = 20
seed = 500
# nAlg = 23

etapa = '2 - AA[dg-arj]'

# base = "cds"
# qp = "QP1"

# bases = ["cds", "dblp"]
bases = ["cd"]
qps = ["qp6"] #Apenas QP6.

for base in bases:
    
    print("Base atual: {0}".format(base))
    
    for qp in qps:
        
        print("QP atual: {0}".format(qp))

        # dirOrig = "../../csv/conjuntosDS/conjuntosDiverg/"
        # dirOrig = "../../csv/conjuntosDS/conjuntosDivergAA/"
#         dirOrig = "../../csv/conjuntosDS/conjuntosDivergAAMenor/"
        dirOrig = "../../csv/conjuntosDS/conjuntosDivergAA/"+base+"/"+qp+"/"
        dirEstat = "../../csv/estatisticas/"+base+"/"+qp+"/"
        estat = dirEstat+"estatisticaDS.csv"


        for k in janelas:

            nao_passaram = 0
            iteracoes = 0

            estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados', 'etapa', 'permutacao'], sep=';')

            ###### Criação do dataframe que armazenará as estatísticas dos conjuntos de treino e teste
        #   estat_conj = pd.DataFrame (index=['algUtl', 'permutacao'], columns=['algUtl','permutacao','tamConjTreino','prctgDup','prctgNaoDup','tamConjTeste'])
            estat_conj = pd.DataFrame (columns=['algUtl','permutacao','tamConjTreino','prctgDup','prctgNaoDup','tamConjTeste', 'passou'])


            arquivos = [] #Adicionado depois

            for _, _, arquivo in os.walk(dirOrig):
                arquivos.extend(arquivo)

            for arq in arquivos:

                if ('_NEW' in arq):# & (iteracoes <= 50):
#                 if ('diverg(10)1_NEW' in arq) & (iteracoes <= 0): #Aqui, Diego!
                    
        #         if ('diverg(10)18_NEW' in arq) & (iteracoes <= 0):
        #         if ('diverg(10)18_NEW' in arq) | ('diverg(10)15_NEW' in arq) & (iteracoes <= 1):
                    print("##################################################################")
                    print("Analisando o arquivo: {0}".format(arq))
                    print("##################################################################")

                    iteracoes += 1

                    num = re.sub(r'diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                    num = num.replace('_NEW.csv','')

                    algUtl = re.sub(r'diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                    algUtl = re.sub(r'\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
                    algUtl = int(algUtl)

                    permutacao = int(num)

                    linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))    

                    ###### Leitura do conjunto de pares conflitantes
                    pc = dirOrig+arq

                    print("pc: {0}".format(pc))

                    pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes

                    cols = list(pc.columns.values)
                    cols.pop(cols.index('duplicata'))
                    pc = pc[['duplicata']+cols]

                    pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
                    pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

                    #Criação da coluna de ENTROPIA

                    #ENTROPIA = −[P+(e) · ln(P+(e)) +P−(e) · ln(P−(e))]

                    #Onde, P+(e) é a fração de casos positivos identificados pelo comitê (|CASOS_POSITIVOS|/|COMITÊ|).

                    #             pc_aa['entropia'] = -(pc_aa['qtdAlg']/nAlg * np.log(pc_aa['qtdAlg']/nAlg) + ((nAlg - pc_aa['qtdAlg'])/nAlg) * np.log((nAlg - pc_aa['qtdAlg'])/nAlg))
                    pc_aa['entropia'] = -(pc_aa['qtdAlg']/algUtl * np.log(pc_aa['qtdAlg']/algUtl) + ((algUtl - pc_aa['qtdAlg'])/algUtl) * np.log((algUtl - pc_aa['qtdAlg'])/algUtl))

                    # pc_aa = pc.loc[:, :'med' ]
                    # pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

                    duplicata = pc_aa.loc[:, 'duplicata' ]

                    #Adicionando a coluna de duplicatas a pc_vetores
                    pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

                    conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

                    ###### 

                    #Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
                    #que aponta o par como possível duplicata 
                    #(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
                    #e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
                    ndup = pc_aa.loc[pc['qtdAlg'] <= qtd_alg_nd]
                    dup = pc_aa.loc[(pc['qtdAlg'] > qtd_alg_nd) & (pc['qtdAlg'] < (algUtl - qtd_alg_nd))]
                    #dup = pc_aa.loc[(pc['qtdAlg'] > qtd_alg_nd) & (pc['qtdAlg'] < (qtd_alg - qtd_alg_nd))]
                    # dup = dup[pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd)]

                    ###### Ordenamento dos pares pela estatística selecionada
        #             ndup = ndup.sort_values(estat_ord) 
        #             dup = dup.sort_values(estat_ord, ascending=False)

                    ###### Ordenamento dos pares pela estatística selecionada 
                    #Modificando a ordenação a fim de fazer com que as janelas deslizem 
                    #a partir do meio do conjunto todo e assim pegar pares da região de incerteza do conjunto todo

        #             ndup = ndup.sort_values(estat_ord) #ndup - pegando do menor para o maior
                    ndup = ndup.sort_values(estat_ord, ascending=False) #ndup - pegando do maior para o menor (INCERTEZA)
                    dup = dup.sort_values(estat_ord) #dup - pegando do menor para o maior (INCERTEZA)
        #             dup = dup.sort_values(estat_ord, ascending=False) #dup - pegando do maior para o menor 

                    ###### Variáveis para verificar se pode deslizar a janela dentro de dup (ou ndup)

        #             deslz_dup = len(dup)/k
        #             deslz_ndup = len(ndup)/k
                    deslz_dup = 0
                    deslz_ndup = 0
                    k_dup = 0
                    k_ndup = 0



                    if (k == 1):

                        deslz_dup = len(dup)
                        deslz_ndup = len(ndup)

                        k_dup = 1
                        k_ndup = 1

                    else:

                        k_dup = int(np.around(len(dup)*k))
                        k_ndup = int(np.around(len(ndup)*k))

                        deslz_dup = len(dup)/k_dup
                        deslz_ndup = len(ndup)/k_ndup

                    print("---At the beginning---")

                    print("len(dup): {0} - len(ndup): {1}".format(len(dup), len(ndup)))
                    print("k: {0} --> k_dup: {1} - k_ndup: {2}".format(k, k_dup, k_ndup))
                    print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))


                    ###### Validação do tamanho para deslizamento

                    #Se não houver espaço suficiente para deslizar as janelas,
                    #a ponto de formar o conjunto mínimo de treinamento
                    if (deslz_dup < tam_min_ct/2) | (deslz_ndup < tam_min_ct/2):
                        pode_passar = False #Teria que ser informado para o usuário definir novo tamanho de janela
                        print("Arquivo: {0} não passou!".format(arq))
                        nao_passaram += 1
                    else:
                        pode_passar = True

                    ###### Criação do dataframe que armazenará o conjunto de treinamento

                    conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

                    ###### Criação do dataframe que armazenará os pares descartados na etada de max_local
                    conj_descart = pd.DataFrame(columns=pc_vetores.columns.values)

                    ###### Povoamento inicial sem aleatoriedade no except

                    orcamento = 200
        #             conj_treino = pd.DataFrame(columns=pc_vetores.columns.values) #LOCAL ORIGINAL DE conj_treino

                    continua = True
                    f1_anterior = 0
                    f1_atual = 0
                    f1_svm = 0
                    f1_dt = 0

                    jan_inic_ndup = 0 
                    jan_fin_ndup = k_ndup

                    jan_inic_dup = 0 
                    jan_fin_dup = k_dup

        #             cont = 0
                    contDup = 0
                    contNDup = 0

                    continua = True
                    unblock_ndup = True #Adicionado para tentar manter o conjunto balanceado
                    unblock_dup = True #Adicionado para tentar manter o conjunto balanceado

                    if (pode_passar):

                        #Povoamento inicial com 20 pares rotulados (10 de cada)
        #                 while (cont < tam_min_ct) & (continua): #Nesse exemplo seriam 20 de cada
                        while ((contDup < tam_min_ct/2) | (contNDup < tam_min_ct/2)) & (continua):

        #                     print('contDup: {0} - contNDup: {1}'.format(contDup, contNDup))
                            reexecuta = True

                            #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                            #Seleciona-se o par com maior valor de similaridade para rotulação

                            deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        #                     while (reexecuta) & (deslz_ndup >= 1):
        #                     while (reexecuta) & (deslz_ndup >= 1) & (unblock_ndup):
                            while (reexecuta) & (deslz_ndup >= 1) & (unblock_ndup) & (contNDup < tam_min_ct/2):

        #                         print("while sendo chamado para ndup!")

        #                         print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))

                                grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                                sel_vet_rot(grupo, 'min', 'ndup', 'o ndup chamou de 20')

                                if not(reexecuta):
                                    contNDup += 1 #Atualização da variável utilizada para monitorar o tamanho do conjunto de treinamento inicial


                            reexecuta = True

                            #SELEÇÃO DA POSSÍVEL DUPLICATA
                            #Seleciona-se o par com maior entropia (mais conflitante) para rotulação

                            deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        #                     while (reexecuta) & (deslz_dup >= 1):
        #                     while (reexecuta) & (deslz_dup >= 1) & (unblock_dup):
                            while (reexecuta) & (deslz_dup >= 1) & (unblock_dup) & (contDup < tam_min_ct/2):

        #                         print("while sendo chamado para dup!")

        #                         print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))

                                grupo = dup.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                                sel_vet_rot(grupo, 'entropia', 'dup', 'o dup chamou de 20')

                                if not(reexecuta):
                                    contDup += 1 #Atualização da variável utilizada para monitorar o tamanho do conjunto de treinamento inicial

                            #Verificação do balanceamento do conjunto de treinamento gerado até então
                            duplicatas = [i for i in conj_treino.duplicata if i == True]
                            duplicatas = duplicatas.count(True)
                            percentDup = float(duplicatas/float(len(conj_treino))*100)
                            percentDup = float("{0:.2f}".format(percentDup))

                            nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
                            nao_duplicatas = nao_duplicatas.count(False)
                            percentNaoDup = float(nao_duplicatas/float(len(conj_treino))*100)
                            percentNaoDup = float("{0:.2f}".format(percentNaoDup))

                            dif_perc = abs(percentDup - percentNaoDup)

                            unblock_dup = True
                            unblock_ndup = True

                            print('contDup: {0} - contNDup: {1}'.format(contDup, contNDup))
                            print('$$$$$Antes de calcular a diferença entre as porcentagens$$$$$')

                            if (dif_perc > 25):
                                print("Diferença entre as porcentagens: {0}".format(dif_perc))
                                print("Porcentagem de dup: {0} - Porcentagem de ndup: {1}".format(percentDup, percentNaoDup))
        #                         input("Enter")
                                if(percentDup > percentNaoDup):
                                    if not (contNDup < tam_min_ct/2): #Se ainda se pode entrar no bloco de atualização de ndup
                                                                     #Do contário, não entraria em tal bloco, esgotaria-se o deslz_ndup
                                                                    #e ainda impediria o preenchimento da quantidade mínima de vetores em dup
                                        unblock_dup = False
                                        unblock_ndup = True
                                else:
                                    if not (contDup < tam_min_ct/2): #Se ainda se pode entrar no bloco de atualização de dup
                                                                    #Do contário, não entraria em tal bloco, esgotaria-se o deslz_dup
                                                                   #e ainda impediria o preenchimento da quantidade mínima de vetores em ndup
                                        unblock_dup = True
                                        unblock_ndup = False


                            #Atualização das janelas
                            jan_inic_ndup = jan_inic_ndup + k_ndup
                            jan_fin_ndup = jan_fin_ndup + k_ndup

                            jan_inic_dup = jan_inic_dup + k_dup
                            jan_fin_dup = jan_fin_dup + k_dup

                            if(deslz_dup < 1) | (deslz_ndup < 1): #Se não houver mais espaço para deslizar as janelas
                                continua = False

        #                     cont += 1 #Atualização da variável utilizada para monitorar o tamanho do conjunto de treinamento inicial

        #                     contNDup += 1 #Atualização da variável utilizada para monitorar o tamanho do conjunto de treinamento inicial

                        #Treina SVM e Decision Tree (justificar o pq) e verifica a média da medida de qualidade (f1, precision...)
                        f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM', 'após os 20')
                        f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT', 'após os 20')

                        f1_atual = (f1_svm + f1_dt)/2
                        std_atual = (std_svm + std_dt)/2
                        cv_atual = (cv_svm + cv_dt)/2

                        print('')
                        print("Tamanho do conjunto de treinamento após os supostos 20: {0}".format(len(conj_treino)))
                        print('')
        #                 print('((contDup < tam_min_ct/2) | (contNDup < tam_min_ct/2)): {0} - (continua): {1}'.format(((contDup < tam_min_ct/2) | (contNDup < tam_min_ct/2)), (continua)))
        #                 print('')
        #                 print('(deslz_dup < 1) :{0} (deslz_ndup < 1):{1}'.format((deslz_dup < 1), (deslz_ndup < 1)))

                        ###### Aplicação das janelas deslizantes após o povoamento incial sem aleatoriedade no except

                        # continua = True
                        max_local = 0
                        igual = 0

            #             print("Início")
            #             print("orcamento: {0}".format(orcamento))
            #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                        while continua:

                            f1_anterior = f1_atual

                            reexecuta = True

                            deslz_ndup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        #                     while (reexecuta) & (deslz_ndup >= 1):
                            while (reexecuta) & (deslz_ndup >= 1) & (unblock_ndup):

        #                         print("while sendo chamado para ndup!")

        #                         print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))

                                grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

        #                         #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
        #                         #Seleciona-se o par com maior valor de similaridade para rotulação
                                sel_vet_rot(grupo, 'min', 'ndup', 'o ndup chamou')

                            reexecuta = True

                            #SELEÇÃO DA POSSÍVEL DUPLICATA
                            #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                            deslz_dup -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        #                     while (reexecuta) & (deslz_dup >= 1):
                            while (reexecuta) & (deslz_dup >= 1) & (unblock_dup):

        #                         print("while sendo chamado para ndup!")

        #                         print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))

                                grupo = dup.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                                sel_vet_rot(grupo, 'entropia', 'dup', 'o dup chamou')


                            #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
                            f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM', 'no deslizamento final')
                            f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT', 'no deslizamento final')

                            f1_atual = (f1_svm + f1_dt)/2
                            std_atual = (std_svm + std_dt)/2
                            cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação

                            cv_atual = cv_atual*100

                            #Atualização das condições de parada
                            #depois de selecionar e rotular a possível duplicata e possível não-duplicata

                            if orcamento > 0 : #Se ainda tem orçamento pra gastar
                                #if cv_atual < 10:
                                if cv_atual <= 5:
                                    if (f1_atual >= f1_anterior) and (igual <= 3): #Selecionam-se novos pares para rotulação
                                        max_local = 0

                                        if f1_atual == f1_anterior:
                                            igual += 1
                                        elif (f1_atual > f1_anterior) and (igual > 0):
                                            igual = 0

                                        continua = True
                                    elif (f1_atual < f1_anterior):
                                        max_local += 1
                                        igual = 0
                                        #Se entrar aqui tem que remover os últimos pares que entraram no conjunto de treinamento

                                        #IMPORTANTE! Fazer com que esses devem sejam extraídos para o conjunto PM, não apenas descartados
                                        conj_descart = conj_descart.append(conj_treino.tail(2)) #Guardando os vetores a serem retirados do conjunto treinamento
                                        conj_treino.drop(conj_treino.tail(2).index,inplace=True)

        #                                 print("f1_atual < f1_anterior")
        #                                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
        #                                 input("Pressione ENTER para continuar")

                                        if (max_local <= 4):
                                            f1_atual = f1_anterior
                                        else:
                                            f1_atual = f1_anterior
        #                                     print('O f1 passou a piorar')
        #                                     print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
        #                                     input("Pressione ENTER para continuar")
                                            continua = False
                                            break
                                    elif (igual > 3): #Se a medida de qualidade for igual à anterior (por três vezes)
        #                                     print('F1 convergiu!') 
        #                                     input("Pressione ENTER para continuar")
                                            continua = False
                                            break

                            else:
                                continua = False
                                break

                            #Verificação do balanceamento do conjunto de treinamento gerado até então
                            duplicatas = [i for i in conj_treino.duplicata if i == True]
                            duplicatas = duplicatas.count(True)
                            percentDup = float(duplicatas/float(len(conj_treino))*100)
                            percentDup = float("{0:.2f}".format(percentDup))

                            nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
                            nao_duplicatas = nao_duplicatas.count(False)
                            percentNaoDup = float(nao_duplicatas/float(len(conj_treino))*100)
                            percentNaoDup = float("{0:.2f}".format(percentNaoDup))

                            dif_perc = abs(percentDup - percentNaoDup)

                            if (dif_perc > 25):
                                if(percentDup > percentNaoDup):
                                    unblock_dup = False
                                    unblock_ndup = True
                                else:
                                    unblock_dup = True
                                    unblock_ndup = False

                            #Atualização das janelas
                            jan_inic_ndup = jan_inic_ndup + k_ndup
                            jan_fin_ndup = jan_fin_ndup + k_ndup

                            jan_inic_dup = jan_inic_dup + k_dup
                            jan_fin_dup = jan_fin_dup + k_dup

                            if(deslz_dup < 1) | (deslz_ndup < 1): #Se não houver mais espaço para deslizar as janelas
                                continua = False

            #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
            #                 print("igual: {0}".format(igual))


                    else: #Se não pode passar, nesses experimentos, ao invés de solicitar que o usuário
                          #determine novo tamanho de janela, é estabelecido um único conjunto por onde
                          #as janelas irão deslizar.

                        print('ENTROU NO ELSE POR NÃO PASSAR')

                        pc_aa = pc_aa.sort_values(estat_ord, ascending = False) 

                        deslz = 0

                        if (k == 1):

                            deslz = len(pc_aa)
                            k = 1

                        else:

                            k = len(pc_aa)*k
                            deslz = len(pc_aa)/k

                        jan_inic_dup = 0 
                        jan_fin_dup = k

                        jan_inic_ndup = len(pc_aa) #Fim de pc_aa
                        jan_fin_ndup = len(pc_aa) - k


                        #se a janela final de ndup for menor que a janela final de dup, para
                        cont = 0
                        #Povoamento inicial com 20 pares rotulados (10 de cada)
                        while (cont < tam_min_ct) & (continua) & (jan_fin_dup < jan_fin_ndup):

                            #Desliza-se a janela
                            reexecuta = True

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

            #                 print("reexecuta: {0} - deslz_ndup >= 1: {1}".format(reexecuta, deslz_ndup >= 1))

                            while (reexecuta) & (deslz >= 1):

            #                     grupo = pc_aa.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe
                                grupo = pc_aa.iloc[jan_fin_ndup:jan_inic_ndup] # three rows of dataframe

                                #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                                #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                                try:

                                    id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                                    #Só passa pra cá se não lançar except
                                    reexecuta = False
                                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                                    vetor_sel = pc_vetores.loc[id_row_sel]
                                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                                    orcamento -= 1


                                except ValueError:

                                    print("Atualizando janelas no except")
                                    print(grupo)
                                    jan_inic_ndup = jan_inic_ndup - k
                                    jan_fin_ndup = jan_fin_ndup - k

                                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

        #                             print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
        #                             print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))

                                    reexecuta = True

                                except:
                                    print('Eita!')

            #                 print("Possível não-duplicata")
            #                 print(vetor_sel)

                            reexecuta = True

                            #SELEÇÃO DA POSSÍVEL DUPLICATA
                            #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            while (reexecuta) & (deslz >= 1):

                                grupo = pc_aa.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                                try:

                                    id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                                    #Só passa pra cá se não lançar except
                                    reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                                    vetor_sel = pc_vetores.loc[id_row_sel]
                                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                                    orcamento -= 1

                                except ValueError:

                                    print("Atualizando janelas no except")
                                    print(grupo)
                                    jan_inic_dup = jan_inic_dup + k
                                    jan_fin_dup = jan_fin_dup + k

                                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                                    print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
                                    print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))


                                    reexecuta = True

                                except:
                                    print('Eita!')

            #                 print("Possível duplicata")
            #                 print(vetor_sel)

                            #Atualização das janelas
                            jan_inic_ndup = jan_inic_ndup - k
                            jan_fin_ndup = jan_fin_ndup - k

                            jan_inic_dup = jan_inic_dup + k
                            jan_fin_dup = jan_fin_dup + k

                            if(deslz < 1): #Se não houver mais espaço para deslizar as janelas
                                continua = False


                            cont += 1


                        #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
                        f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM', 'após os 20 quando ENTROU NO ELSE POR NÃO PASSAR')
                        f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT', 'após os 20 quando ENTROU NO ELSE POR NÃO PASSAR')

                        f1_atual = (f1_svm + f1_dt)/2
                        std_atual = (std_svm + std_dt)/2
                        cv_atual = (cv_svm + cv_dt)/2

                        ###### Aplicação das janelas deslizantes após o povoamento incial sem aleatoriedade no except


                        # continua = True
                        max_local = 0
                        igual = 0

            #             print("Início")
            #             print("orcamento: {0}".format(orcamento))
            #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                        while continua:

                            f1_anterior = f1_atual

                            reexecuta = True

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            while (reexecuta) & (deslz_ndup >= 1):
                                grupo = pc_aa.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                                #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                                #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                                try:

                                    id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                                    #Só passa pra cá se não lançar except
                                    reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                                    vetor_sel = pc_vetores.loc[id_row_sel]
                                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                                    orcamento -= 1

                                except ValueError:

                                    print("Atualizando janelas no except")
                                    jan_inic_ndup = jan_inic_ndup - k
                                    jan_fin_ndup = jan_fin_ndup - k

                                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                                    reexecuta = True

                                except:
                                    print('Eita!')

            #                 print("Possível não-duplicata")
            #                 print(vetor_sel)

                            reexecuta = True

                            #SELEÇÃO DA POSSÍVEL DUPLICATA
                            #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            while (reexecuta) & (deslz >= 1):

                                grupo = pc_aa.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                                try:

                                    id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                                    #Só passa pra cá se não lançar except
                                    reexecuta = False
                                    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                                    vetor_sel = pc_vetores.loc[id_row_sel]
                                    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                                    orcamento -= 1

                                except ValueError:

                                    print("Atualizando janelas no except")
                                    jan_inic_dup = jan_inic_dup + k
                                    jan_fin_dup = jan_fin_dup + k

                                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                                    reexecuta = True

                                except:
                                    print('Eita!')

            #                 print("Possível duplicata")
            #                 print(vetor_sel)

                            #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
                            f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM', 'quando ENTROU NO ELSE POR NÃO PASSAR')
                            f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT', 'quando ENTROU NO ELSE POR NÃO PASSAR')

                            f1_atual = (f1_svm + f1_dt)/2
                            std_atual = (std_svm + std_dt)/2
                            cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação

                            cv_atual = cv_atual*100


                            if orcamento > 0 : #Se ainda tem orçamento pra gastar
                                #if cv_atual < 10:
                                if cv_atual <= 5:
                                    if (f1_atual >= f1_anterior) and (igual <= 3): #Selecionam-se novos pares para rotulação
                                        max_local = 0

                                        if f1_atual == f1_anterior:
                                            igual += 1
                                        elif (f1_atual > f1_anterior) and (igual > 0):
                                            igual = 0

                                        continua = True
                                    elif (f1_atual < f1_anterior):
                                        max_local += 1
                                        igual = 0
                                        #Se entrar aqui tem que remover os últimos pares que entraram no conjunto de treinamento

                                        conj_descart = conj_descart.append(conj_treino.tail(2)) #Guardando os vetores a serem retirados do conjunto treinamento
                                        conj_treino.drop(conj_treino.tail(2).index,inplace=True)

            #                             print("f1_atual < f1_anterior")
            #                             print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))

                                        if (max_local <= 4):
                                            f1_atual = f1_anterior
                                        else:
                                            f1_atual = f1_anterior
            #                                 print('O f1 passou a piorar')
            #                                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                                            continua = False
                                            break
                                    elif (igual > 3): #Se a medida de qualidade for igual à anterior (por três vezes)
            #                                 print('F1 convergiu!') 
                                            continua = False
                                            break

                            else:
                                continua = False
                                break

                            #Atualização das janelas
                            jan_inic_ndup = jan_inic_ndup - k
                            jan_fin_ndup = jan_fin_ndup - k

                            jan_inic_dup = jan_inic_dup + k
                            jan_fin_dup = jan_fin_dup + k

                            if(deslz < 1): #Se não houver mais espaço para deslizar as janelas
                                continua = False

                    print("---At the end---")
                    print("deslz_dup: {0} - deslz_ndup: {1}".format(deslz_dup, deslz_ndup))
                    print("Tamanho do conjunto de treinamento: {0}".format(len(conj_treino)))
                    ##### Salvando os arquivos de treino e teste

        #                 dirDest = "../../csv/conjuntosDS/treinoTeste/"

        #             dirDest = "../../csv/conjuntosDS/treinoTeste-k"+str(k)+"/"
                    dirDest = "../../csv/conjuntosDS/treinoTeste/"+base+"/"+qp+"/DgArj-k"+str(k)+"/"

                    abordagem = 'DS'

                    #Verdadeiros positivos constantes no conjunto de pares descartados
                    tp_adc = [i for i in conj_descart.duplicata if i == True]
                    tp_adc = tp_adc.count(True)

                    #Verdadeiros negativos constantes no conjunto de pares descartados
                    tn_adc = [i for i in conj_descart.duplicata if i == False]
                    tn_adc = tn_adc.count(False)

                    print("tp_adc: {0} - tn_adc: {1}".format(tp_adc,tn_adc))

                    iteracao = 1
                    inspecoesManuais = orcamento_orig - orcamento

                    duplicatas = [i for i in conj_treino.duplicata if i == True]
                    duplicatas = duplicatas.count(True)
                    percentDup = float(duplicatas/float(len(conj_treino))*100)
                    percentDup = float("{0:.2f}".format(percentDup))
        #             print("{0}% de duplicatas.".format(str(percentage)[:5]))

                    nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
                    nao_duplicatas = nao_duplicatas.count(False)
                    percentNaoDup = float(nao_duplicatas/float(len(conj_treino))*100)
                    percentNaoDup = float("{0:.2f}".format(percentNaoDup))
        #             print("{0}% de nao-duplicatas.".format(str(percentage2)[:5]))

            #             print("linhaAtual")
            #             print(linhaAtual)

                    da = linhaAtual['da'].item()
                    dm = duplicatas + tp_adc
                    ndm = nao_duplicatas + tn_adc

                    tp = float(linhaAtual['tp'].item() + dm)
                    fp = float(linhaAtual['fp'].item())
                    tn = float(linhaAtual['tn'].item())# + ndm) #Retirado
                    fn = float(linhaAtual['fn'].item() - dm) #Adicionado

                    precision = tp/(tp+fp)
                    recall = tp/(tp+fn)
                    fmeasure = 2*((precision*recall)/(precision+recall))

                        #Adicionando valor à última linha
                    estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                        'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                        'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais, precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])

        #             dirDest = "../../csv/conjuntosDS/treinoTeste-k"+str(k)+"/" #COmentei agora, DIego!
                #         dirDest = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
                #         dirDest = "./arqResult/csv/conjuntosDS/conjuntosDiverg/treinoTeste/"

                        #algUtl = str(algUtl).replace('.0','')
                    algUtl = str(algUtl)

                    geraTrainSet(conj_treino, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

                    indicesCT = conj_treino.index.values.tolist()

                    indicesDSCRT = conj_descart.index.values.tolist() ##VERIFICAR SE ISSO PROCEDE

        #             print("ANTES DE GERAR O CONJUNTO GERAL")

        #             print("pc_vetores: {0} - conj_treino: {1} - conj_descart: {2}".format(len(pc_vetores), len(conj_treino), len(conj_descart)))
        #             print("conj_teste deve ser: {0}".format((len(pc_vetores) - (len(conj_treino) + len(conj_descart)))))

        #             geral = pd.concat([pc_vetores,conj_treino,conj_descart]) #Concatenando pc_vetores e conj_treino
                    geral = pd.concat([pc_vetores,conj_treino]) #Concatenando pc_vetores e conj_treino

        #             print("DEPOIS DE GERAR O CONJUNTO GERAL")
        #             print("geral: {0}".format(len(geral)))

                    #Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
                    conj_teste = geral.drop(indicesCT, axis='rows')

        #             print("DEPOIS DE DROPAR O QUE ESTAVA NO CONJUNTO DE TREINAMENTO")
        #             print("conj_teste: {0}".format(len(conj_teste)))

        #             conj_teste = conj_teste.drop(indicesDSCRT, axis='rows') #PQ CARGAS D'ÁGUAS CRIEI ESSA LINHA?

        #             print("DEPOIS DE DROPAR O QUE ESTAVA NO CONJUNTO DE DESCARTES")
        #             print("conj_teste: {0}".format(len(conj_teste)))
                
                    geraTestSet(conj_teste, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')

        #             estats_conj = [[algUtl, permutacao, len(conj_treino), percentDup, percentNaoDup,len(conj_teste)]]

                    estat_conj.loc[len(estat_conj.index)] = [algUtl, permutacao, len(conj_treino), percentDup, percentNaoDup, len(conj_teste), pode_passar]
#                 arq = ''
        #     print("Estatísticas")
        #     print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))
        #     print(conj_treino)

            ############################################################################################################################
            #Estatísticas
            ############################################################################################################################

            # if (pode_passar):

            ###### Criação do dataframe que armazenará as estatísticas dos conjuntos de treino e teste
        #     estat_conj = pd.DataFrame(estat_conj_list, columns=['algoritmosUtilizados','permutacao','tamConjTreino','prctgDup','prctgNaoDup','tamConjTeste'])

        #     estat_conj = estat_conj.append(estat_conj_dic, ignore_index=True)

            #Para voltar o dataframe ao normal
            estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'etapa', 'permutacao'])

            estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

            estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
            estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

        #     dirEst = "../../csv/"

            # Diretório para Windows
            # dirEst = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\\"
            # dirEst = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/"CA


            # Diretório para Linux
            # dirEst = "./arqResult/csv/"

            estatisticas.to_csv(dirEstat+'estatisticaDS2-DgArj-k'+str(k)+'.csv', sep=';', index=False)   

            estat_conj.to_csv(dirEstat+'estatisticasConjuntos-DgArj-k'+str(k)+'.csv', sep=';', index=False)   

            if(iteracoes >= 3000):

                print("Não passaram: {0}".format(nao_passaram))



Base atual: cd
QP atual: qp6
##################################################################
Analisando o arquivo: diverg(10)10_NEW.csv
##################################################################
pc: ../../csv/conjuntosDS/conjuntosDivergAA/cd/qp6/diverg(10)10_NEW.csv
---At the beginning---
len(dup): 159 - len(ndup): 419
k: 1 --> k_dup: 1 - k_ndup: 1
deslz_dup: 159 - deslz_ndup: 419
contDup: 1 - contNDup: 1
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Diferença entre as porcentagens: 100.0
Porcentagem de dup: 100.0 - Porcentagem de ndup: 0.0
contDup: 2 - contNDup: 2
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Diferença entre as porcentagens: 100.0
Porcentagem de dup: 100.0 - Porcentagem de ndup: 0.0
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 155 - deslz_ndup: 416
contDup: 3 - contNDup: 3
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Diferença entre as porcentagens: 66.66
Porcentagem de dup: 83.

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 75 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 74 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 73 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 72 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 71 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 70 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 69 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 68 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 67 - deslz_ndup: 329
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 66 - deslz_ndup: 329
Atualizando janelas depois de 

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 93 - deslz_ndup: 273
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 91 - deslz_ndup: 273
contDup: 3 - contNDup: 3
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Diferença entre as porcentagens: 33.34
Porcentagem de dup: 66.67 - Porcentagem de ndup: 33.33
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 91 - deslz_ndup: 271
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 91 - deslz_ndup: 270
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 91 - deslz_ndup: 269
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 91 - deslz_ndup: 268
contDup: 4 - contNDup: 4
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
contDup: 5 - contNDup: 5
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Diferença entre as porcentagens: 40.0
Porcentagem de dup: 70.0 

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 66 - deslz_ndup: 177
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 66 - deslz_ndup: 176
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 66 - deslz_ndup: 175
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 64 - deslz_ndup: 172
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 63 - deslz_ndup: 170
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 59 - deslz_ndup: 165
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 57 - deslz_ndup: 162
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 52 - deslz_ndup: 156
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 51 - deslz_ndup: 154
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 51 - deslz_ndup: 153
Atualizando janelas depois de 

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 45
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 44
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 43
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 42
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 41
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 40
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 39
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 38
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19 - deslz_ndup: 37
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 18 - deslz_ndup: 35
Atualizando janelas depois de dropar tod

...

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 65.0 - deslz_ndup: 75.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 64.0 - deslz_ndup: 75.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 61.0 - deslz_ndup: 73.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 59.0 - deslz_ndup: 72.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 55.0 - deslz_ndup: 69.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 52.0 - deslz_ndup: 67.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 51.0 - deslz_ndup: 67.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 50.0 - deslz_ndup: 67.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 49.0 - deslz_ndup: 67.4
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 46.0 - deslz_ndup: 65.4


Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 38.0 - deslz_ndup: 66.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 36.0 - deslz_ndup: 65.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 35.0 - deslz_ndup: 65.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 34.0 - deslz_ndup: 65.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 33.0 - deslz_ndup: 65.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 31.0 - deslz_ndup: 64.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 30.0 - deslz_ndup: 64.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 29.0 - deslz_ndup: 64.16666666666667
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 27.0 - deslz_ndup: 6

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 1.5 - deslz_ndup: 51.0
---At the end---
deslz_dup: 0.5 - deslz_ndup: 50.0
Tamanho do conjunto de treinamento: 63
tp_adc: 0 - tn_adc: 0
##################################################################
Analisando o arquivo: diverg(15)4_NEW.csv
##################################################################
pc: ../../csv/conjuntosDS/conjuntosDivergAA/cd/qp6/diverg(15)4_NEW.csv
---At the beginning---
len(dup): 187 - len(ndup): 893
k: 0.01 --> k_dup: 2 - k_ndup: 9
deslz_dup: 93.5 - deslz_ndup: 99.22222222222223
contDup: 1 - contNDup: 1
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 92.5 - deslz_ndup: 96.22222222222223
contDup: 2 - contNDup: 2
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
contDup: 3 - contNDup: 3
$$$$$Antes de calcular a diferença entre as porcentagens$$$$$
contDup: 4 - contNDup: 4
$$$$$Ant

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 45.0 - deslz_ndup: 57.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 41.0 - deslz_ndup: 54.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 40.0 - deslz_ndup: 54.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 37.0 - deslz_ndup: 52.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 35.0 - deslz_ndup: 51.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 34.0 - deslz_ndup: 51.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 33.0 - deslz_ndup: 51.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 31.0 - deslz_ndup: 50.83333333333333
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 30.0 - deslz_ndup: 5

Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 20.0 - deslz_ndup: 51.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 19.0 - deslz_ndup: 51.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 18.0 - deslz_ndup: 51.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 17.0 - deslz_ndup: 51.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 14.0 - deslz_ndup: 49.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 13.0 - deslz_ndup: 49.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 12.0 - deslz_ndup: 49.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 11.0 - deslz_ndup: 49.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 10.0 - deslz_ndup: 49.125
Atualizando janelas depois de dropar todos os elementos do grupo
deslz_dup: 9.0 - 